In [4]:
import os
from subprocess import Popen, PIPE

aprimo_path = "/mnt/aprimo"

files_in_path = os.listdir(os.path.join(aprimo_path))

In [12]:
for f in files_in_path:
    if f.startswith("aprimo") and f.endswith("txt"):
        print Popen("file -i {}".format(os.path.join(aprimo_path, f)).split(), stdout=PIPE).communicate ()[0]

/mnt/aprimo/aprimo_activity_utf.txt: text/plain; charset=utf-8

/mnt/aprimo/aprimo-digital-asset.txt: text/plain; charset=unknown-8bit

/mnt/aprimo/aprimo_integrated_campaign.txt: application/octet-stream; charset=binary

/mnt/aprimo/aprimo_activity.txt: text/plain; charset=iso-8859-1



In [28]:
activity = open("/mnt/aprimo/aprimo_activity.txt", "rb").readlines()

In [30]:
esl = lambda x: x.decode("ISO-8859-1").encode("utf-8").split("¬")

In [31]:
activity = [esl(row) for row in activity if len(esl(row)) == 44]

In [43]:
print len(activity)
print Popen("wc -l {}".format("/mnt/aprimo/aprimo_activity.txt").split(), stdout=PIPE).communicate()[0]

290408
291497 /mnt/aprimo/aprimo_activity.txt



### Testing Success (99.62%)

In [47]:
campaign = open("/mnt/aprimo/aprimo_integrated_campaign.txt", "rb").readlines()

In [48]:
campaign = [esl(row) for row in campaign if len(esl(row)) == 50]

In [50]:
print len(campaign)
print Popen("wc -l {}".format("/mnt/aprimo/aprimo_integrated_campaign.txt").split(), stdout=PIPE).communicate()[0]

106866
106891 /mnt/aprimo/aprimo_integrated_campaign.txt



In [52]:
import pickle

campaign_cols = pickle.load(open(os.path.join(aprimo_path, "campaign_columns.p"), "rb"))
campaign = spark.createDataFrame(campaign, campaign_cols)

In [71]:
campaign.select(campaign.columns[:8]).sample(False, .10, 88).show(5)

+----------------------+-----------+----------+-------------------+-----+-------------------+------+---------------+
|integrated_campaign_id|proposal_id|mapping_id|         begin_date|brand|     classification|client|  currency_code|
+----------------------+-----------+----------+-------------------+-----+-------------------+------+---------------+
|                  1098|           |          |2012-05-01 00:00:00|     |HP Global Marketing|      |USD (US Dollar)|
|                 10980|           |          |2012-11-02 00:00:00|     |HP Global Marketing|      |USD (US Dollar)|
|                 10996|           |          |2010-06-01 00:00:00|     |HP Global Marketing|      |USD (US Dollar)|
|                 11283|           |          |2013-08-07 00:00:00|     |HP Global Marketing|      |USD (US Dollar)|
|                 11652|           |          |2013-04-30 00:00:00|     |HP Global Marketing|      |USD (US Dollar)|
+----------------------+-----------+----------+-----------------

In [98]:
campaign.write.parquet("/mnt/aprimo/clean/aprimo-integrated-campaign.parquet")

In [96]:
class AprimoParser(object):
    
    def __init__(self, path_to_file, encoding="ISO-8859-1", delim="\t", expected_cols=False):
        self.path_to_file = path_to_file
        self.encoding = encoding
        self.delim = delim
        self.expected_cols = expected_cols
        self.read_in = False
    
    def __getitem__(self, key):
        if not self.read_in:
            return self.read()[key]
        return self.data[key]
            
    def read(self):
        self.data = open(self.path_to_file, "rb").readlines()
        des = lambda x: x.decode(self.encoding).encode("utf-8").split(self.delim)
        if self.expected_cols:
            self.data = [des(row) for row in self.data if len(esl(row)) == self.expected_cols]
        else:
            self.data = [des(row) for row in self.data]
        self.read_in = True
        return self
    
    def to_spark(self, columns):
        if not self.read_in:
            self.data = spark.createDataFrame(self.read(), columns)
            return self
        self.data = spark.createDataFrame(self.data, columns)
        return self
    

camp = AprimoParser(os.path.join(aprimo_path, "aprimo_integrated_campaign.txt"), delim="¬", expected_cols=50)

In [101]:
asset_ = AprimoParser(os.path.join(aprimo_path, "aprimo-digital-asset.txt"), delim="¬", expected_cols=56)

In [105]:
asset_ = asset_.to_spark(pickle.load(open(os.path.join(aprimo_path, "asset_columns.p"))))

In [108]:
asset_.data.write.parquet("/mnt/aprimo/clean/aprimo-digital-asset.parquet")